In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
crises_df = pd.read_csv('data/african_crises.csv')
print(crises_df.isnull().sum())

In [ ]:
crises = crises_df[['systemic_crisis', 'domestic_debt_in_default', 
                    'currency_crises', 'inflation_crises', 
                    'banking_crisis']]

ccs = crises_df['cc3'].unique()

for cc in ccs:
    years = crises_df.loc[crises_df['cc3'] == cc]['year']
    inflation = crises_df.loc[crises_df['cc3'] == cc]['inflation_annual_cpi']
    
    plt.plot(years, inflation, label=cc)

plt.yscale('log')
plt.show()

In [ ]:
for cc in ccs:
    years = crises_df.loc[crises_df['cc3'] == cc]['year']
    print(cc, min(years))

In [ ]:
def get_color(cc3):
    
    portuguese_colonies = ['AGO']
    english_colonies = ['EGY', 'KEN', 'NGA', 'ZAF', 'ZMB', 'ZWE', 'MUS']
    french_colonies = ['DZA', 'CIV', 'CAF', 'MAR', 'TUN']
    
    if cc3 in portuguese_colonies:
        return 'g'
    elif cc3 in english_colonies:
        return 'r'
    elif cc3 in french_colonies:
        return 'b'
    

crises_cols = ['systemic_crisis', 'domestic_debt_in_default', 
                'currency_crises', 'inflation_crises', 
                'banking_crisis']

crises_df_after_1957 = crises_df.loc[crises_df['year'] >= 1957]

bool_crises_df = crises_df_after_1957[['systemic_crisis', 'domestic_debt_in_default', 
                                       'currency_crises', 'inflation_crises', 
                                       'banking_crisis', 'cc3']]

# Replace values in banking_crisis with boolean values
bool_crises_df = bool_crises_df.replace({'banking_crisis': {'crisis': 1, 'no_crisis': 0}})

ccs = crises_df['cc3'].unique()

# Iterate over columns that signify crises
for crisis in crises_cols:
    crisis_counts = []
    # For each country save the sum (amount of crisis years)
    for cc in ccs:
        crisis_counts.append(bool_crises_df[crisis].loc[bool_crises_df['cc3'] == cc].sum())
        
    # Sort country codes and counts based on counts
    cc_sorted, counts_sorted = zip(*[(x, y) for y, x in sorted(zip(crisis_counts, ccs))])
    
    # Give each country a color
    colors = [get_color(cc) for cc in cc_sorted]
    plt.title(crisis)
    plt.bar(cc_sorted, counts_sorted, color=colors)
    plt.show()